In [10]:
#!/usr/bin/env python

import pygame,sys, math
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
from pygame import *
from sys import exit
from math import *

width = 1366
height = 768
clock = pygame.time.Clock()

translateX = 0.0
translateY = 0.0
translateZ = 0.0

cameraX = 0.0
cameraY = 60.0
cameraZ = 0.0

mouseAngle = [45,-90]

angleX = 0.0
angleY = 0.0
angleZ = 0.0

currentPlanet = 0
eyeX = 0.0
eyeZ = 0.0
eyeY = 0.0

scaleWsp = 1.0
matrixScale = [scaleWsp,0,0,0,0,scaleWsp,0,0,0,0,scaleWsp,0,0,0,0,1]
matrixTranslate = [1,0,0,0,0,1,0,0,0,0,1,0,translateX,translateY, translateZ,1]
latitude = 45
longitude = 45

stateOrbity = True

Sun = [0.0, 2.32, 0.0, 0.0,0.0,0.0]
Mercury = [5.0,0.1, 1.0, 0.11,0.0,0.0]
venus = [6.0, 0.2, 0.5,0.48,0.0,0.0]
earth = [7.0, 0.2, 0.25, 0.04,0.0,0.0]
mars = [8.0, 0.1, 0.125, 0.048,0.0,0.0]
jupiter = [11.0, 1.0, 0.05, 0.01,0.0,0.0]
saturn = [13.0, 0.8, 0.0025,0.02,0.0,0.0]
uranus = [15.0, 0.6, 0.0012,0.034,0.0,0.0]
neptune = [17.0, 0.4, 0.0006, 0.032,0.0,0.0]
planety = [Sun, Mercury, venus, earth, mars, jupiter, saturn, uranus, neptune] #Wiem, ze Slonce nie jest planeta :P
images = ['sunmap.jpg', 'mercurymap.jpg', 'venusmap.jpg', 'earthmap.jpg', 'marsmap.jpg', 'jupitermap.jpg', 'saturnmap.jpg', 'uranusmap.jpg', 'neptunemap.jpg']
textures = range(len(images))


def InitGL(Width, Height):
    global cameraX, cameraY, cameraZ, translateX, translateY, translateZ, squaresAmount
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)
    glDepthFunc(GL_LESS)
    glEnable(GL_DEPTH_TEST)
    glShadeModel(GL_SMOOTH)
    glEnable(GL_BLEND)
    glEnable(GL_TEXTURE_2D)
    
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45.0, float(Width)/float(Height), 0.1, 400.0)
    gluLookAt(cameraX, cameraY, cameraZ, 0,0,0, 0,1,0)
    
    glNewList(1,GL_COMPILE)
    glBegin(GL_LINES)
    for i in range(360):
        glVertex3f(sin(radians(i)), 0,cos(radians(i)))
        glVertex3f(sin(radians((i+1))), 0,cos(radians((i+1))))
        
    glEnd()
    glEndList()
    
    glNewList(2,GL_COMPILE)
    glBegin(GL_QUADS)
    for j in range(latitude):
        for i in range(longitude):
    
            glTexCoord2f(float((latitude - i)) / float(latitude),
                         float((longitude - j)) / float(longitude))
    
            
            glVertex3f(sin(radians(i * (360 / longitude))) * cos(radians(j * (180 / latitude))),
                       cos(radians(j * (180 / longitude))),
                       sin(radians(i * (360 / longitude))) * sin(radians(j * (180 / latitude))))
            
            
            glTexCoord2f(float((latitude - (i + 1))) / float(latitude),
                         float((longitude - j)) / float(longitude))
            
            
            glVertex3f(
                sin(radians((i + 1) * (360 / longitude))) * sin(radians(j * (180 / latitude))),
                cos(radians(j * (180 / longitude))),
                cos(radians((i + 1) * (360 / latitude))) * sin(radians(j * (180 / latitude)))) #for orbit

    glEnd()
    
    glEndList()
    
    glMatrixMode(GL_MODELVIEW)

def LoadTextures():
    global textures, images
    
    load_textures = []
    
    textures = glGenTextures(len(images))
    for tex in range(len(images)):
        load_textures.append([[],[],[]])
        load_textures[tex][0] = pygame.image.load(images[tex])
        load_textures[tex][1] = load_textures[tex][0].get_width()
        load_textures[tex][2] = load_textures[tex][0].get_height()
        load_textures[tex][0] = pygame.image.tostring(load_textures[tex][0], 'RGBA', True)
        
        glBindTexture(GL_TEXTURE_2D, textures[tex])
        glTexParameteri(GL_TEXTURE_2D,GL_TEXTURE_MIN_FILTER, GL_LINEAR_MIPMAP_LINEAR)
        glTexParameteri( GL_TEXTURE_2D,GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        gluBuild2DMipmaps(GL_TEXTURE_2D, 4, load_textures[tex][1], load_textures[tex][2], GL_RGBA, GL_UNSIGNED_BYTE, load_textures[tex][0])


def Scene(Width, Height):
    global cameraX, cameraY, cameraZ, translateX, translateY, translateZ, planety, eyeX, eyeZ, eyeY
    
    glViewport(0, 0, Width, Height)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45.0, float(Width)/float(Height), 0.1, 400.0)
    gluLookAt(cameraX, cameraY, cameraZ, eyeX,eyeY,eyeZ, 0,1,0)
    
    glMatrixMode(GL_MODELVIEW)
    
def Draw():
    global matrixScale, scaleWsp, matrixTranslate, matrixRotate, angleX, angleY, angleZ, planety, translateX, translateY, translateZ, stateOrbity
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    
    
    #glColor3f(0,0,1)
    if stateOrbity:
        for i in range(len(planety)):
            
            scaleWsp = planety[i][0]
            matrixScale = [scaleWsp,0,0,0,0,scaleWsp,0,0,0,0,scaleWsp,0,0,0,0,1]
            glMultMatrixf(matrixScale)
            glCallList(1)
            glLoadIdentity()
    
    #glColor3f(1,0,0)
    for i in range(len(planety)):
        glBindTexture(GL_TEXTURE_2D, textures[i])
        scaleWsp = planety[i][1]
        translateY = 0.0
        translateX = (planety[i][0])*sin(radians(planety[i][4]))
        translateZ = (planety[i][0])*cos(radians(planety[i][4]))

    
        matrixScale = [scaleWsp,0,0,0,0,scaleWsp,0,0,0,0,scaleWsp,0,0,0,0,1]
        matrixTranslate = [1,0,0,0,0,1,0,0,0,0,1,0,translateX,translateY, translateZ,1]
        glMultMatrixf(matrixTranslate)
        glMultMatrixf(matrixScale)  
        glCallList(2)
        glLoadIdentity()
        
        planety[i][4] += planety[i][2]
        planety[i][5] += planety[i][3]
    
    pygame.display.flip()

def Keys():
    global cameraX, cameraY, cameraZ, currentPlanet, planety, eyeX, eyeY, eyeZ, mouseAngle, stateOrbity
    
    for i in pygame.event.get():
        if i.type == QUIT or (i.type==KEYDOWN and i.key==K_ESCAPE):
            pygame.quit()
            sys.exit()
        if i.type == KEYDOWN:
            if i.key == K_o:
                stateOrbity = not stateOrbity
    
    key = pygame.key.get_pressed()
    if key[K_UP]:
        cameraX += (sin(radians(mouseAngle[0]))*cos(radians(mouseAngle[1])))/10
        cameraZ += (cos(radians(mouseAngle[0]))*cos(radians(mouseAngle[1])))/10
        cameraY += sin(radians(mouseAngle[1]))/10
    elif key[K_DOWN]:
        cameraX -= (sin(radians(mouseAngle[0]))*cos(radians(mouseAngle[1])))/10
        cameraZ -= (cos(radians(mouseAngle[0]))*cos(radians(mouseAngle[1])))/10
        cameraY -= sin(radians(mouseAngle[1]))/10
    
    
  
    eyeX = cameraX + (sin(radians(mouseAngle[0]))*cos(radians(mouseAngle[1])))
    eyeZ = cameraZ + (cos(radians(mouseAngle[0]))*cos(radians(mouseAngle[1])))
    eyeY = cameraY + sin(radians(mouseAngle[1]))    
    
def main():
    pygame.init()
    screen = pygame.display.set_mode((width,height),DOUBLEBUF|OPENGL)
    pygame.event.set_grab(1)
    InitGL(width, height)
    LoadTextures()
    while True:
        Keys()
        clock.tick(60)
        Scene(width, height)
        Draw()

main()



SystemExit: 